In [13]:
import torch
from torch import nn

print(torch.__version__)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

2.4.0+cu121


'cuda'

# The Individuals' Class
- The Hyperparamters should be passed to the constructor in a way that is both convenient for GP and for PyTorch.
- I think I want to define a class for one `nn.Sequential` 2d-block
    - All possible instances should be concatenable with all possible instances
- Then, an individual is built from the concatenation of many such blocks, plus data preparation and final f.c. layer

In [14]:
''' 
define a λ nn.Module that creates an nn layer from a given function
this is handy for nn.Sequential usage '''
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x):
        return self.func(x)
    
'''
create a function to reshape the (28x28) image input data '''
def preprocess(x):
    return x.view(-1, 1, 28, 28)

class Sequential_block_2d(nn.Module):
    def __init__(self, in_channels: int,
                 out_channels: int, # the number of output neurons after the full block
                 conv_kernel_size: int = 3,
                 conv_stride: int = 1,
                 conv_padding: int = 1,
                 pool_kernel_size: int = 2,
                 pool_stride: int = 2,
                 pool_padding: int = 0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=conv_kernel_size,
                      stride=conv_stride,
                      padding=conv_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool_kernel_size,
                         stride=pool_stride,
                         padding=pool_padding)
        )
    def forward(self, x):
        return self.block(x)
    
testBlock = Sequential_block_2d(in_channels=1,out_channels=5).to(device)
testBlock

Sequential_block_2d(
  (block): Sequential(
    (0): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [ ]:
# class for image classification individuals
class NN_individual(nn.Module):
    def __init__(self, in_dimensions: int,
                 out_dimensions: int,
                 blocks_2d: list[Sequential_block_2d],
                 fin_res: int = 5): # output dimension of the final max pooling 
                                    # producing size (fin_res * fin_res)
        super().__init__()
        self.blocks_2d = blocks_2d
        # add a final max pool
        # Why? Because then torch handles the dimensions through the "adaptive"ness
        self.last_max_pool_adaptive = nn.AdaptiveAvgPool2d((fin_res, fin_res))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = fin_res * fin_res * blocks_2d[-1].out_channels,
                      out_features = out_dimensions))
    def forward(self, x):
        for i in range(len(self.blocks_2d)):
            x = self.blocks_2d(x)
        return self.classifier(
            self.last_max_pool_adaptive(x))